In [ ]:
import numpy as np

In [ ]:
def matrix_product(A, B):
    if type(A[0]) != list:
        A = [A]

    if len(A[0]) != len(B):
        raise ValueError("행렬 곱 연산이 불가능합니다.")

    result = [[0 for _ in range(len(B[0]))] for _ in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            dot_product = 0
            for k in range(len(A[0])):
                dot_product += A[i][k] * B[k][j]
            result[i][j] = dot_product
    return result

In [ ]:
##########주어짐###########
#입력
X=[[i+0.1,i] for i in range(1000)] #한 시점에 특징이 2가지인 데이터 1000개
#print("X:",np.array(X).shape)
#print(X)
#배치 구현 <- simple_lstm 참고하여 바꿔야
BatchSize = int(input("배치 사이즈:"))
X=X[:BatchSize]
#print("X:",np.array(X).shape)
#print(X)
timeSize = int(input("한번에 고려할 시점 수:")) #한번에 고려할 시점의 개수
newX=[]
for i in range(len(X)-timeSize+1): # [0 1 2 3 4]를 2시점 씩 고려하면 [0 1] [1 2] [2 3] [3 4] 총 4가지가 되므로 
    smallX=[]
    for j in range(timeSize):
        smallX.append(X[i+j])
    newX.append(smallX)
X = newX
print("X:",np.array(X).shape)
print(X)
input_len = len(X)  #데이터의 개수만큼 target과 예측값이 있어야 하므로

#target
T=[[i//10] for i in range(input_len)] # 임의로 첫번째 값을 10으로 나눈 몫으로 정함 , 타겟이 마지막 층이라 원소가 1개인 target 값이 input_len만큼있음
print("T:",np.array(T).shape)
print(T)

#셔플수행하면 좋으나 그냥지나감

In [ ]:
outputQ = input("LSTM 한 층의 입력하나요(Y/N)?:") #층이 하나면 그냥 target의 종류, 층이 여러개면 정해줘야 함. 현재는 단일 모듈이므로 target의 종류로 정함
count_list=[] # 타겟의 종류 수 카운트 => layer output의 크기를 정하기 위해서(위의 ? 정함) lstm 층이 여러개면 lstm 모듈마다 output_size 정해야
cnt = 0
for now in T:
    if now in count_list:
        pass
    else:
        cnt += 1
        count_list.append(now)
output_size = cnt #w의 output 크기
if(outputQ == "N"):
    output_size = int(input("output_size:")) 
print(output_size)

In [ ]:
##########작성###########
#state초기화
Y = [[[0 for i in range(output_size)] for i in range(timeSize)] for _ in range(input_len)] # 실제 값은 X의 입력 만큼 있어야 하므로
C = [[[0 for i in range(output_size)] for i in range(timeSize)] for _ in range(input_len)] # X은 시점 수 만큼 있어야 하므로
print("Y:",np.array(Y).shape)
print(Y)
print("C:",np.array(C).shape)
print(C)

In [ ]:
#parameter초기화
input_feature = len(X[0][0]) # 한 시점에서 데이터의 특징 수 <= X와 w의 행렬곱을 수행해야하므로 X가 (시점수,2)이므로 w는 (2,?)가 되야함
print(input_feature)

In [ ]:
#parameter 초기화
import random
#가중치 초기화: 가우시안 초기화 방법,  He 초기화 (He Initialization) 또는 Xavier 초기화 (Glorot Initialization)도 존재
w_matrix = []
v_matrix = []
b_matrix = []
for _ in range(4): ## 망각 게이트,입력 게이트,새로운 기억, 출력 게이트 총 4개에서 업데이트 해야함
    w0=[[random.gauss(0,1) for i in range(output_size)] for j in range(input_feature)] # X는 (시점 수, 특징 수)이므로 w 는 (특징수, output_size) 
    #print(np.array(w0).shape)
    v0=[[random.gauss(0,1) for i in range(output_size)]for j in range(output_size)]  # Y는 (시점 수, 특징 수)이므로 w 는 (특징수, output_size)
    #print(np.array(v0).shape)
    b0=[[random.gauss(0,1) for i in range(output_size)]for j in range(timeSize)] 
    #print(np.array(b0).shape)
    w_matrix.append(w0)
    v_matrix.append(v0)
    b_matrix.append(b0)
print('X')
print(np.array(X).shape)
print('w_matrix')
print(np.array(w_matrix).shape)
print('Y')
print(np.array(Y).shape)
print('v_matrix')
print(np.array(v_matrix).shape)
################################
print('b_matrix')
print(np.array(b_matrix).shape)


In [ ]:
now = 1
nowtime = 1
print("============  matrix product (Xw)  ============\n")
#print(np.matmul(X[now],w_matrix[0]).shape) .matnul는 2차원만 연산 가능
#print(np.matmul(X[now],w_matrix[0]))
print("-----------------Using Numpy------------------")
print(np.dot(X[now],w_matrix[0]).shape)
print(np.dot(X[now],w_matrix[0]))
print("\n----------------What I made------------------")
print(np.array(matrix_product(X[nowtime],w_matrix[0])).shape)
print(np.array(matrix_product(X[nowtime],w_matrix[0])))


In [ ]:
print("==================")
print("Yv")
print(np.dot(Y[now],v_matrix[0]).shape)
print(np.dot(Y[now],v_matrix[0]))
print(np.array(matrix_product(Y[nowtime],v_matrix[0])).shape)
print(np.array(matrix_product(Y[nowtime],v_matrix[0])))

In [ ]:
def element_sum(arr1, arr2):
    if len(arr1) != len(arr2)  or len(arr1[0]) != len(arr2[0]):
        raise ValueError("Input arrays must have the same shape")
    #print(np.array(arr1))
    #print(np.array(arr2))
    result = []
    for i in range(len(arr1)):
        row = []
        for j in range(len(arr1[0])):
            row.append(arr1[i][j] + arr2[i][j])
        result.append(row)
    
    return result

In [ ]:
print("========   elemental sum  (Xw+Yv+b) ==========\n")
print("-----------------Using Numpy------------------")
print((np.dot(X[now],w_matrix[0])+np.dot(Y[now],v_matrix[0])+b_matrix[0]).shape)
print((np.dot(X[now],w_matrix[0])+np.dot(Y[now],v_matrix[0])+b_matrix[0]))
print("\n----------------What I made------------------")
Xw = matrix_product(X[nowtime],w_matrix[0])
Yv = matrix_product(Y[nowtime],v_matrix[0])
Xw_Yv_b = element_sum(element_sum(Xw,Yv),b_matrix[0])
print(np.array(Xw_Yv_b).shape)
print(np.array(Xw_Yv_b))


In [ ]:
def activation(function,x):
    e = 2.718281828459045 #자연상수:2.71828182845904523536
    result = []
    if(function == 'Sigmoid'):
        for i in range(len(x)):
            row = []
            for j in range(len(x[0])):
                now_x = x[i][j]
                row.append((e**now_x)/((e**now_x)+1)) #C언어에서는 **이 없어 **도 구현
            result.append(row)
        return result
    if(function == 'Tanh'):
        for i in range(len(x)):
            row = []
            for j in range(len(x[0])):
                now_x = x[i][j]
                row.append((2/(1+(e**(-2*now_x))))-1)
            result.append(row)
        return result
    if(function == 'ReLU'):
        for i in range(len(x)):
            row = []
            for j in range(len(x[0])):
                now_x = x[i][j]
                if (now_x<=0):
                    row.append(0)
                else:
                    row.append(now_x)
            result.append(row)
        return result

In [ ]:
print("======== Activate Fuction (Sigmoid) ==========\n")
print("-----------------Using Numpy------------------")
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
print(sigmoid(np.array(Xw_Yv_b)))
print("\n----------------What I made------------------")
print(np.array(activation('Sigmoid',Xw_Yv_b)))

In [ ]:
print("======== Activate Fuction (Tanh) ==========\n")
print("-----------------Using Numpy------------------")
npX=np.tanh(Xw_Yv_b)
print(npX)
print("\n----------------What I made------------------")
print(np.array(activation('Tanh',Xw_Yv_b)))

In [ ]:
###################################LSTM###################################
nowtime = 1
pasttime = nowtime-1
#forgot gate
print("===========forgot gate============")
Xw = matrix_product(X[nowtime],w_matrix[0])
Yv = matrix_product(Y[pasttime],v_matrix[0])
Xw_Yv_b = element_sum(element_sum(Xw,Yv),b_matrix[0])
fGate = activation('Sigmoid',Xw_Yv_b)
print(np.array(fGate).shape)
print(np.array(fGate))

In [ ]:
#input gate
print("===========input gate============")
Xw = matrix_product(X[nowtime],w_matrix[1])
Yv = matrix_product(Y[pasttime],v_matrix[1])
Xw_Yv_b = element_sum(element_sum(Xw,Yv),b_matrix[1])
iGate = activation('Sigmoid',Xw_Yv_b)
print(np.array(iGate).shape)
print(np.array(iGate))

In [ ]:
#new input
print("===========new input============")
Xw = matrix_product(X[nowtime],w_matrix[2])
Yv = matrix_product(Y[pasttime],v_matrix[2])
Xw_Yv_b = element_sum(element_sum(Xw,Yv),b_matrix[2])
newIn = activation('Tanh',Xw_Yv_b)
print(np.array(newIn).shape)
print(np.array(newIn))

In [ ]:
#output gate
print("===========output gate============")
Xw = matrix_product(X[nowtime],w_matrix[3])
Yv = matrix_product(Y[pasttime],v_matrix[3])
Xw_Yv_b = element_sum(element_sum(Xw,Yv),b_matrix[3])
oGate = activation('Sigmoid',Xw_Yv_b)
print(np.array(oGate).shape)
print(np.array(oGate))

In [ ]:
def element_multi(arr1, arr2):
    if len(arr1) != len(arr2) or len(arr1[0]) != len(arr2[0]):
        raise ValueError("Input arrays must have the same shape")
    #print(np.array(arr1))
    #print(np.array(arr2))
    result = []
    for i in range(len(arr1)):
        row = []
        for j in range(len(arr1[0])):
            row.append(arr1[i][j] * arr2[i][j])
        result.append(row)
    return result

In [ ]:
print("========   elemental multiplication  ==========\n")
print("-----------------Using Numpy------------------")
np_C = np.array(C)
np_F = np.array(fGate)
np_CF = np_C[pasttime] * np_F
print(np_CF.shape)
print(np_CF)
print("\n----------------What I made------------------")
CF = element_multi(C[pasttime], fGate)
print(np.array(CF).shape)
print(np.array(CF))


In [ ]:
print(np.array(C[nowtime]).shape)
print(np.array(C[nowtime]))

In [ ]:
print(np.array(Y[nowtime]).shape)
print(np.array(Y[nowtime]))

In [ ]:
#State
C[nowtime] = element_sum(element_multi(C[pasttime], fGate),element_multi(iGate, newIn))
Y[nowtime] = element_multi(oGate,activation('Tanh',C[nowtime]))

In [ ]:
print(np.array(C[nowtime]).shape)
print(np.array(C[nowtime]))

In [ ]:
print(np.array(Y[nowtime]).shape)
print(np.array(Y[nowtime]))